In [1]:
# Imports the required libraries
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Imports the machine learning modules
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm


# Imports the data reporting modules
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# Inputs the CSV Path for the three (3) antecedent models
# Load the Oil Returns Data into "csv_path_1"
csv_path_1 = Path("./Resources/technical_output.csv")
csv_path_2 = Path("./Resources/fundamental_output.csv")
csv_path_3 = Path("./Resources/macro_output.csv")

In [6]:
# Loads the CSV files
model_one_output = pd.read_csv(csv_path_1, index_col="Date", parse_dates=True, infer_datetime_format=True)
model_two_output = pd.read_csv(csv_path_2, index_col="Date", parse_dates=True, infer_datetime_format=True)
model_three_output = pd.read_csv(csv_path_3, index_col="Date", parse_dates=True, infer_datetime_format=True)

# Concatenates all three into a single DataFrame (called "model_inputs")
model_inputs = pd.concat([model_one_output, model_two_output, model_three_output],  axis= "columns", join="inner")

# Renames columns to generic variables
column_names = ["x_1 linear", "x_2 linear", "x_3 linear"]
model_inputs.columns = column_names

# Creates three (3) additional inputs that squares the  original data
model_inputs["x_1 exponential"] = model_inputs["x_1 linear"] * model_inputs["x_1 linear"]
model_inputs["x_2 exponential"] = model_inputs["x_2 linear"] * model_inputs["x_2 linear"]
model_inputs["x_3 exponential"] = model_inputs["x_3 linear"] * model_inputs["x_3 linear"]

# Initializes the new Signal column
model_inputs['Signal'] = 0.0

# When Actual Returns ("x_1 linear") are greater than or equal to 0, generates signal to buy stock long
model_inputs.loc[(model_inputs["x_1 linear"] >= 1), 'Signal'] = 1

# When Actual Returns ("x_1 linear") are less than 1, generates signal to sell oil
model_inputs.loc[(model_inputs["x_1 linear"] < 1), 'Signal'] = -1

# Calculates the strategy returns and add them to the signals_df DataFrame
model_inputs['Strategy Returns'] = model_inputs['x_1 linear'] * model_inputs['Signal'].shift()

# Prints cleaned DataFrame
model_inputs = model_inputs.dropna()
model_inputs

,x_1 linear,x_2 linear,x_3 linear,x_1 exponential,x_2 exponential,x_3 exponential,Signal,Strategy Returns
Date,,,,,,,,
2020-06-02,1,1,1,1,1,1,1.0,1.0
2020-06-03,1,-1,1,1,1,1,1.0,1.0
2020-06-04,1,1,1,1,1,1,1.0,1.0
2020-06-05,1,1,1,1,1,1,1.0,1.0
2020-06-08,1,-1,1,1,1,1,1.0,1.0
...,...,...,...,...,...,...,...,...
2022-03-25,1,1,1,1,1,1,1.0,1.0
2022-03-28,1,-1,1,1,1,1,1.0,1.0
2022-03-29,1,1,1,1,1,1,1.0,1.0


### Data Preparation: "Train, Test, Split" & "Oversampler" ###

In [7]:
# Creates the target set selecting the Signal column and assiging it to y
y = model_inputs["Signal"]

# Creates the model inputs by dropping Signal & Strategy Returns columns and assigning it to X
X = model_inputs.drop(columns=["Signal", "Strategy Returns"])

# Splits the data using train_test_split and assigns a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Checks the y values to determine if oversampling is needed
y.value_counts()

 1.0    427
-1.0     33
Name: Signal, dtype: int64

In [8]:
# Instantiates the random oversampler model with a random_state parameter of 1
random_oversampler = RandomOverSampler(random_state=1)

# Fits the original training data to the random_oversampler model
X_resampled, y_resampled = random_oversampler.fit_resample(X_train, y_train)

# Checks the resamped y values
y_resampled.value_counts()

 1.0    322
-1.0    322
Name: Signal, dtype: int64

In [9]:
# Creates a StandardScaler instance
scaler = StandardScaler()

# Applys the scaler model to fit the X-train data
X_scaler = scaler.fit(X_resampled)

# Transforms the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_resampled)
X_test_scaled = X_scaler.transform(X_test)

### Model 1: Linear Regression ###

In [10]:
# Creates a LogisticRegression model and trains it on the X_train_scaled and y_resampled
regression_model = LogisticRegression()
regression_model.fit(X_train_scaled, y_resampled)

# Uses the model you trained to predict using X_test_scaled
y_pred = regression_model.predict(X_test_scaled)

# Creates a classification report to evaluate performance and saves it
regression_testing_report = classification_report(y_test, y_pred, digits=4)

# Prints out a classification report to evaluate performance
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

        -1.0     1.0000    1.0000    1.0000        10
         1.0     1.0000    1.0000    1.0000       105

    accuracy                         1.0000       115
   macro avg     1.0000    1.0000    1.0000       115
weighted avg     1.0000    1.0000    1.0000       115



### Model 2: Random Forest Classification ####

In [11]:
# Creates a RandomForestClassifier model and trains it on the X_train_scaled
forest_model = RandomForestClassifier(random_state=0)

# Uses the model you trained to predict using X_test_scaled
forest_model.fit(X_train_scaled, y_resampled)
y_pred = forest_model.predict(X_test_scaled)

# Creates a classification report to evaluate performance and saves it
forest_testing_report = classification_report(y_test, y_pred, digits=4)

# Prints out a classification report to evaluate performance
print(forest_testing_report)

              precision    recall  f1-score   support

        -1.0     1.0000    1.0000    1.0000        10
         1.0     1.0000    1.0000    1.0000       105

    accuracy                         1.0000       115
   macro avg     1.0000    1.0000    1.0000       115
weighted avg     1.0000    1.0000    1.0000       115



### Model 3: Random Forest Classification ####

In [12]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_resampled)
 
# Uses the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Creates a classification report to evaluate performance and saves it
svm_testing_report = classification_report(y_test, svm_pred, digits=4)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

        -1.0     1.0000    1.0000    1.0000        10
         1.0     1.0000    1.0000    1.0000       105

    accuracy                         1.0000       115
   macro avg     1.0000    1.0000    1.0000       115
weighted avg     1.0000    1.0000    1.0000       115

